<h1 style="text-align:center">Análise Exploratória de Dados de Notícias de Jornal Web</h1>

<div align="center"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/8f/Correio_Braziliense.svg/2560px-Correio_Braziliense.svg.png" width="500px" height="50px" alt="Logo Correio Braziliense"></img></div>

<p align='justify'>As notícias de jornal web são coletadas diariamente por webscraping a partir do site de notícias <a href="https://www.correiobraziliense.com.br/">Correio Braziliense</a>. É um jornal brasileiro com sede em Brasília - DF, fundado em 21 de abril de 1960 por Assis Chateaubriand, juntamente com a inauguração da cidade e da TV Brasília. O nome veio do histórico <a href="https://pt.wikipedia.org/wiki/Correio_Braziliense_(1808)">Correio Brasiliense ou Armazém Literário</a>, editado em Londres a partir de 1808 por Hipólito José da Costa <cite><a href="https://pt.wikipedia.org/wiki/Correio_Braziliense">[Wikipedia]</a></cite>.</p>

<p align='justify'>Como premissa para as análises e organização do projeto, é utilizada a metodologia <a href="https://www.datascience-pm.com/crisp-dm-2/">Cross Industry Standard Process for Data Mining (CRISP-DM)</a>.</p>

<h2 style="text-align:left">Entendimento do Negócio</h2>

<p align='justify'>O conjunto de dados em análise, refere-se a notícias diárias de um jornal de Brasília-DF chamado Correio Braziliense, onde por meio da técnica de webscraping, é coletado as notícias diárias desse jornal, uma tabela dispõe de informações das URL's das notícias, tema, e data da extração, a outra tabela dispõe de informações da notícias em si, data da publicação, autor, título e o texto da notícias.</p>

<h3 style="text-align:left">Dicionário de dados</h3>
<p align='justify'>A seguir é demonstrado os campos (colunas) e sua respectiva descrição a que se refere tal dado em cada tabela construída no banco de dados.</p>

<h4 style="text-align:left">TB_JORNAIS</h4>
<p align='justify'>Tabela com informações básicas do jornal.</p>
    
|Campo|Descrição|
|:----|:--------|
|ID_JORNAL|Identificador único do jornal|
|DESC_JORNAL|Nome do jornal|
|URL_JORNAL|URL principal do jornal|

<h4 style="text-align:left">TB_URL_NOTICIAS</h4>
<p align='justify'>Tabela com informações das URL's da notícias extraídas do jornal.</p>

|Campo|Descrição|
|:----|:--------|
|ID_NOTICIA|Identificador único da notícia|
|DATA_EXTRACAO|Data da extração da url da notícia|
|DESC_JORNAL|Nome do jornal|
|DESC_TEMA|Nome do tema da notícia|
|URL_NOTICIA|URL da notícia|
|FLAG_EXTRAIDA|Flag (True ou False) se já foi extraído o texto da notícia|

<h4 style="text-align:left">TB_TEXTO_NOTICIAS</h4>
<p align='justify'>Tabela com informações das notícias extraídas do jornal.</p>

|Campo|Descrição|
|:----|:--------|
|ID_NOTICIA|Identificador único da notícia|
|URL|URL da notícia|
|DATA_PUBLICACAO|Data e hora da publicação da notícia|
|AUTOR|Autor(es) da notícia|
|TITULO|Título (manchete) da notícia|
|TEXTO|Texto da notícia|

<h2 style="text-align:left">Entendimento dos Dados</h2>

<p align='justify'>Nesta etapa é realizada a exploração dos dados, com o objetivo de entender as características da base de dados e possíveis ajustes necessários a serem realizados antes da etapa de visualização.</p>

In [4]:
#!pip install wordcloud -q

In [5]:
# Importa Bibliotecas.
import os
import sys
from dotenv import load_dotenv
import time
import re
from datetime import date

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# Apresenta os dados no notebook
%matplotlib inline

import seaborn as sns
# Estilo dos gráficos
sns.set_style("darkgrid")

from wordcloud import WordCloud

import psycopg2 
import sqlalchemy               
from sqlalchemy import create_engine
from sqlalchemy import text

load_dotenv()

# Parametros do Pandas, limitando a quantidade máxima e a largura das colunas.
pd.set_option('max_colwidth', 5000)

In [6]:
# Credenciais banco
host=os.getenv('BD_POSTGRESQL_HOST')
port=os.getenv('BD_POSTGRESQL_PORT')
dbname=os.getenv('BD_POSTGRESQL_DBNAME')
user=os.getenv('BD_POSTGRESQL_USE')
password=os.getenv('BD_POSTGRESQL_PASSWORD')

In [7]:
path = "C:/Users/Usuario/Documents/GitHub/monitoring-webscraping-newspaper/dados/"

In [11]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df_url = pd.read_sql(text('''select * from tb_url_noticias;'''), conn)

# Proporção dos dados em linhas e colunas
print(df_url.shape)

conn.close()

(18706, 6)


In [12]:
# Verificando os metadados da base
df_url.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18706 entries, 0 to 18705
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID_NOTICIA     18706 non-null  int64 
 1   DATA_EXTRACAO  18706 non-null  object
 2   DESC_JORNAL    18706 non-null  object
 3   DESC_TEMA      18706 non-null  object
 4   URL_NOTICIA    18706 non-null  object
 5   FLAG_EXTRAIDA  18706 non-null  bool  
dtypes: bool(1), int64(1), object(4)
memory usage: 749.1+ KB


In [13]:
# Verificando os tipos de dados.
print("Tipos de dados e sua frequência\n{}".format(df_url.dtypes.value_counts()))

Tipos de dados e sua frequência
object    4
int64     1
bool      1
dtype: int64


In [14]:
# Observando a abse de dados
df_url.head()

,ID_NOTICIA,DATA_EXTRACAO,DESC_JORNAL,DESC_TEMA,URL_NOTICIA,FLAG_EXTRAIDA
0,3,2022-09-24,correiobraziliense,home,https://www.correiobraziliense.com.br/politica/2022/09/5039139-tse-identifica-rs-605-milhoes-em-doacoes-suspeitas.html,True
1,4,2022-09-24,correiobraziliense,home,https://www.correiobraziliense.com.br/cidades-df/2022/09/5039108-confira-a-agenda-dos-candidatos-ao-gdf-para-este-sabado.html,True
2,261,2022-10-05,correiobraziliense,opiniao,https://www.correiobraziliense.com.br//opiniao/2022/10/5041502-artigo-o-brasil-mudou.html,True
3,1639,2022-10-13,correiobraziliense,home,https://www.correiobraziliense.com.br/cbforum/testedopezinho.html,True
4,2156,2022-10-17,correiobraziliense,tecnologia,https://www.correiobraziliense.com.br/tecnologia/2022/10/5044373-ia-vence-os-obstaculos.html,True


In [15]:
# Verificação da existência de dados ausentes/faltantes
df_url.isnull().sum()

ID_NOTICIA       0
DATA_EXTRACAO    0
DESC_JORNAL      0
DESC_TEMA        0
URL_NOTICIA      0
FLAG_EXTRAIDA    0
dtype: int64

In [16]:
# Verificando dados nulos por registro e retornando a maior quantidade.
df_url.isna().sum(axis=1).max()

0

In [19]:
df_url['DATA_EXTRACAO'].value_counts()

2023-01-25    309
2022-10-05    304
2022-10-13    296
2022-10-10    295
2022-11-29    197
             ... 
2022-10-23     74
2023-03-26     71
2022-10-16     66
2022-10-29     64
2022-09-24     45
Name: DATA_EXTRACAO, Length: 141, dtype: int64

In [20]:
df_url['DESC_JORNAL'].value_counts()

correiobraziliense    18624
Name: DESC_JORNAL, dtype: int64

In [ ]:
df_url['DESC_JORNAL'].value_counts()

In [33]:
df_url_tema = df_url[['DESC_TEMA', 'DATA_EXTRACAO', 'ID_NOTICIA']].groupby(by=['DESC_TEMA', 'DATA_EXTRACAO']).count()
df_url_tema

ID_NOTICIA
DESC_TEMA DATA_EXTRACAO            
brasil    2022-10-05             20
          2022-10-06             14
          2022-10-07             20
          2022-10-09             10
          2022-10-10             23
...                             ...
turismo   2022-10-05             20
          2022-10-13             20
          2023-02-06              2
          2023-02-28              1
          2023-03-29              1

[1647 rows x 1 columns]

In [34]:
df_url_tema.index

MultiIndex([(    'brasil', '2022-10-05'),
            (    'brasil', '2022-10-06'),
            (    'brasil', '2022-10-07'),
            (    'brasil', '2022-10-09'),
            (    'brasil', '2022-10-10'),
            (    'brasil', '2022-10-11'),
            (    'brasil', '2022-10-12'),
            (    'brasil', '2022-10-13'),
            (    'brasil', '2022-10-14'),
            (    'brasil', '2022-10-15'),
            ...
            ('tecnologia', '2023-03-27'),
            ('tecnologia', '2023-03-28'),
            ('tecnologia', '2023-03-29'),
            ('tecnologia', '2023-03-30'),
            ('tecnologia', '2023-03-31'),
            (   'turismo', '2022-10-05'),
            (   'turismo', '2022-10-13'),
            (   'turismo', '2023-02-06'),
            (   'turismo', '2023-02-28'),
            (   'turismo', '2023-03-29')],
           names=['DESC_TEMA', 'DATA_EXTRACAO'], length=1647)

In [ ]:
df_filmes_qtd_ator = pd.DataFrame({'DESC_TEMA':filmes_qtd_ator.index, 'DATA_EXTRACAO':filmes_qtd_ator.values}).sort_values(by="qtd", ascending=False, ignore_index=True)

In [26]:
# Consultando dados anteriores no BD.
engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')
conn = engine.connect()

df_txt = pd.read_sql(text('''select * from tb_texto_noticias 
                            where "DATA_PUBLICACAO" between '2023-02-01-0000:000:00-10800' 
                            AND '2023-02-30-0000:00:00-10800';'''), conn)
df_txt.shape

conn.close()

In [27]:
df_txt.head(1)

,ID_NOTICIA,URL,DATA_PUBLICACAO,AUTOR,TITULO,TEXTO
0,12798,https://www.correiobraziliense.com.br/politica/2023/02/5073967-sede-da-oab-rj-e-esvaziada-apos-ameaca-de-bomba.html,2023-02-15-0317:01:00-10800,"{""'Agência Estado'""}",Sede da OAB-RJ é esvaziada após ameaça de bomba,"O prédio onde funciona a sede da Ordem dos Advogados do Brasil no Rio de Janeiro (OAB-RJ), no centro da capital fluminense, foi esvaziado às pressas por volta do meio-dia desta quarta-feira, após serem encontradas duas cartas anônimas afirmando que uma bomba explodiria dentro do imóvel. O Esquadrão Antibombas da Polícia Civil foi acionado, iniciou uma varredura pelo prédio e, até a publicação desta reportagem, não havia encontrado nenhum artefato explosivo.\n\nA carta afirma que ""uma bomba foi instalada no edifício (...) e está programada para explodir neste dia 15. (...) Os efeitos serão catastróficos"". Segundo a mensagem, seria o início de uma série de atentados ""em diversos órgãos"" para ""acabar com essa política idiota de exclusão nepotismo vantagens pessoais anuidades caras vaidade absurda reacionarismo barato de uma burguesia hipócrita que se preocupa somente com a indicação pelo quinto constitucional"" (sic).\n\nO quinto constitucional está previsto no artigo 94 da Constituição brasileira e determina que um quinto das vagas de determinados tribunais do País seja preenchido por advogados e membros do Ministério Público, e não por juízes de carreira.\n\nO texto segue: ""O poder a todo o custo (sic) tem seu preço. A OAB pagará preço caro por adotar essa política de manter em seus quadros a direita extremista misógena (sic) homofóbica racista"". E conclui: ""Esvaziem o prédio porque os efeitos serão impactantes. Muitos serão feridos ou perderão suas vidas.""\n\nDois exemplares da carta foram deixados no prédio - um dentro de um elevador e outro no banheiro do 7º andar - e encontrados por funcionários terceirizados responsáveis pela limpeza do imóvel. Segundo a OAB-RJ, existem câmeras de segurança no prédio, cujas imagens serão pesquisadas pela polícia.\n\nNo momento em que as cartas foram encontradas e o prédio foi esvaziado, estava ocorrendo uma cerimônia de entrega de carteiras da OAB para advogados aprovados em concurso recente.\n\nAté o início da tarde desta quarta-feira não havia suspeita de quem pudesse ter distribuído as cartas ou planejado o atentado.\n\nO formato de distribuição de notícias do Correio Braziliense pelo celular mudou. A partir de agora, as notícias chegarão diretamente pelo formato Comunidades, uma das inovações lançadas pelo WhatsApp. Não é preciso ser assinante para receber o serviço. Assim, o internauta pode ter, na palma da mão, matérias verificadas e com credibilidade. Para passar a receber as notícias do Correio, clique no link abaixo e entre na comunidade:\n\nApenas os administradores do grupo poderão mandar mensagens e saber quem são os integrantes da comunidade. Dessa forma, evitamos qualquer tipo de interação indevida.\n\nQuer ficar por dentro sobre as principais notícias do Brasil e do mundo? Siga o Correio Braziliense nas redes sociais. Estamos no Twitter, no Facebook, no Instagram, no TikTok e no YouTube. Acompanhe!"


In [29]:
file_name = "tb_texto_noticias"
extensao = ".txt"

# df_txt.to_csv(path+file_name+"_tab"+"_utf8"+extensao, sep='\t', encoding='utf-8')
df_txt.to_csv(path+file_name+extensao, sep='|', encoding='utf-8', index=False)
# df_txt.to_csv(path+file_name+"_bardup"+"_utf8"+extensao, sep=';', encoding='utf-8')

# df_txt.to_csv(path+file_name+"_tab"+"_latin1"+extensao, sep='\t', encoding ='latin1')
# df_txt.to_csv(path+file_name+extensao, sep='|', encoding ='not utf-8', index=False)
# df_txt.to_csv(path+file_name+"_bardup"+"_latin1"+extensao, sep=';', encoding ='latin1')